In [149]:
from joblib import dump, load

model_filename = './Model/music_classification_svm_model.joblib'
load_model = load(model_filename)

In [150]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from googleapiclient.discovery import build
import config
import numpy as np

spotify_client_id = config.SPOTIFY_CLIENT_ID
spotify_client_secret = config.SPOTIFY_CLIENT_SECRET
youtube_api_key = config.YOUTUBE_API_KEY

client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
youtube = build('youtube', 'v3', developerKey=youtube_api_key)

In [151]:
playlist_id = {
    # 'JP': '37i9dQZEVXbKqiTGXuCOsB',
    # 'MY': '37i9dQZEVXbKcS4rq3mEhp',
    # 'KR': '37i9dQZEVXbJZGli0rRP3r',
    # 'SG': '37i9dQZEVXbN66FupT0MuX',
    # 'TW': '37i9dQZEVXbMVY2FDHm6NN',
    # 'US': '37i9dQZEVXbLp5XoPON0wI',
    # 'UK': '37i9dQZEVXbMwmF30ppw50',
    # 'HK': '37i9dQZEVXbMdvweCgpBAe',
    # 'GLOBAL': '37i9dQZEVXbNG2KDcFcKOF',
    'Random': '2aoayTW7wl1RMg2fX57a6u',
    'Random1': '10MUi4bRKEktYVTrssZKFH',
    'Random3': '5WrWDlu0MrmzFHcfzqquPR',
    'Random4': '0DwuLF3PUhgxFzA8XfIcHy'
}


In [152]:
def get_playlist_tracks(sp, playlist_id):
    tracks = sp.playlist_tracks(playlist_id)
    all_tracks = tracks['items']
    while tracks['next']:
        tracks = sp.next(tracks)
        all_tracks.extend(tracks['items'])

    return all_tracks

In [153]:
top_tracks_by_country = {}

for country, playlist_id in playlist_id.items():
    top_tracks_by_country[country] = get_playlist_tracks(sp, playlist_id)

In [154]:
top_tracks_by_country_filtered = {
    country: [track_info for track_info in tracks if track_info['track']['album'].get('artists')]
    for country, tracks in top_tracks_by_country.items()
}

In [155]:
tracks_data = {}

for country_code, country_tracks in top_tracks_by_country_filtered.items():
    for track_info in country_tracks:
        track = track_info['track']
        track_id = track['id']
        
        if track_id not in tracks_data:
            # If this is the first time we're seeing this track, create a new entry
            tracks_data[track_id] = {
                'trackID': track_id,
                'title': track['name'],
                'artistID': track['artists'][0]['id'],  # Assuming first artist is the primary
                'albumID': track['album']['id'],
                'duration': track['duration_ms'],
                'releaseDate': track['album']['release_date'],
                'spotifyURL': track['external_urls']['spotify'],
                'popularity_by_country': {country_code: track['popularity']}
            }
        else:
            # If we've seen this track before, just update the popularity for the new country
            tracks_data[track_id]['popularity_by_country'][country_code] = track['popularity']

In [156]:
# Splitting track IDs into batches
track_ids = [track['trackID'] for track in tracks_data.values()]
batch_size = 50  # Adjust batch size as needed
track_id_batches = [track_ids[i:i + batch_size] for i in range(0, len(track_ids), batch_size)]

# Initialize a dictionary to store the audio features for each track
audio_features_data = {}

# Spotipy client has been set up previously as 'sp'
for batch in track_id_batches:
    # Fetch audio features for the batch of track IDs
    features_batch = sp.audio_features(batch)
    
    for features in features_batch:
        track_id = features['id']
        if features:  # If features were found
            audio_features_data[track_id] = {
                'tempo': features['tempo'],
                'key': features['key'],
                'mode': features['mode'],
                'loudness': features['loudness'],
                'valence': features['valence']
                # Add any other audio features you're interested in
            }
            # Incorporate the fetched audio features into the tracks_data
            tracks_data[track_id]['audio_features'] = audio_features_data[track_id]
        else:
            # Handle the case where audio features are not found
            print(f"No audio features found for track ID {track_id}")


In [157]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df = pd.DataFrame.from_dict(audio_features_data, orient='index')
df

,tempo,key,mode,loudness,valence
0nt1LAZjaTjrf30S8hdRDU,172.121,4,1,-6.773,0.271
0F02KChKwbcQ3tk4q1YxLH,137.130,7,1,-7.295,0.399
4zWz6OmFpyXIgQCxAS5yQb,73.947,2,1,-6.207,0.397
4KJx4IOvCU2isprmXio9Xk,147.916,0,1,-10.306,0.345
4hYHaROtjB6YREh6AgW3fd,135.977,0,1,-7.191,0.150
...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,139.941,1,1,-8.355,0.877
4NI8oSMRCJdpJPFux5G92u,127.845,1,1,-5.705,0.161
2tgIvnkdDSBYsNBPGeffLJ,122.831,2,1,-8.455,0.222
2gjE7iURty9r45rlgp6JT1,132.273,9,1,-8.061,0.181


In [158]:
def key_mode_category(row):
    if row['mode'] == 1:  # Major
        return f"{row['key']} Major"
    else:  # Minor
        return f"{row['key']} Minor"
    
df['key_mode'] = df.apply(key_mode_category, axis=1)

In [159]:
high_valence_threshold = 0.7  # Above this value, tracks are categorized as 'Happy'.
low_valence_threshold = 0.3   # Below this value, tracks are categorized as 'Sad' or 'Angry'.

# Categorize tracks based on valence
df['valence_emotion'] = df['valence'].apply(lambda v: 'Happy' if v > high_valence_threshold 
                                    else ('Sad/Angry' if v < low_valence_threshold 
                                          else 'Neutral'))

# Count the number of tracks in each category
emotion_counts = df['valence_emotion'].value_counts()

# Print the counts
print(emotion_counts)


valence_emotion
Neutral      111
Sad/Angry     81
Happy         12
Name: count, dtype: int64


In [160]:
def revise_categorization(tempo, loudness, key, mode, valence):
    # Define the criteria for 'Happy'
    if valence > 0.5 and tempo > 100:
        return 'Happy'
    # Define the criteria for 'Sad'
    elif valence < 0.5 and tempo < 100 and mode == 0:
        return 'Sad'
    # Define the criteria for 'Angry'
    elif loudness > -5 and tempo > 120:
        return 'Angry'
    # Define the criteria for 'Neutral'
    elif 0.4 < valence < 0.6:
        return 'Neutral'
    else:
        return 'Uncategorized'

# Apply the revised categorization function to the DataFrame
df['feature_emotion'] = df.apply(lambda row: revise_categorization(row['tempo'], row['loudness'], row['key'], row['mode'], row['valence']), axis=1)

In [161]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
features = df[['tempo', 'loudness', 'key', 'mode', 'valence']]
scaled_features = scaler.fit_transform(features)

In [162]:
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)  # You may choose to optimize the number of clusters
df['cluster'] = kmeans.fit_predict(scaled_features)
centroids = scaler.inverse_transform(kmeans.cluster_centers_)
cluster_features = pd.DataFrame(centroids, columns=['tempo', 'loudness', 'key', 'mode', 'valence'])
cluster_features

,tempo,loudness,key,mode,valence
0,102.457392,-7.131329,5.810127,1.000000e+00,0.465494
1,98.018600,-16.323200,2.100000,1.000000e+00,0.235270
2,145.440278,-7.356835,5.025316,1.000000e+00,0.264557
3,126.107833,-6.734639,5.138889,2.220446e-16,0.406375


In [163]:
# For example, if a centroid has a high valence and a high tempo, you might interpret that cluster as 'Happy'
for i, centroid in enumerate(cluster_features.values):
    tempo, loudness, key, mode, valence = centroid
    print(f"Cluster {i}:")
    if valence > 0.7 and tempo > 120:
        print("Might represent Happy tracks")
    elif valence < 0.4 and tempo < 100:
        print("Might represent Sad tracks")
    elif loudness > -5 and tempo > 120:
        print("Might represent Angry tracks")
    elif 0.4 <= valence <= 0.6:
        print("Might represent Neutral tracks")
    else:
        print("Might be more complex or require further analysis")
    print()


Cluster 0:
Might represent Neutral tracks

Cluster 1:
Might represent Sad tracks

Cluster 2:
Might be more complex or require further analysis

Cluster 3:
Might represent Neutral tracks



In [164]:
cluster_to_emotion = {
    0: 'Angry',
    1: 'Sad',
    2: 'Neutral',
    3: 'Happy'
}

In [165]:
df['cluster_emotion'] = df['cluster'].map(cluster_to_emotion)

In [166]:
def combine_emotions(row):
    # Define the scoring for each method
    valence_score = 3
    feature_score = 1
    cluster_score = 2

    # Special handling for 'Sad/Angry' with 'Uncategorized' feature emotion
    if row['valence_emotion'] == 'Sad/Angry' and row['feature_emotion'] == 'Uncategorized':
        if row['cluster_emotion'] in ['Sad', 'Angry']:
            return row['cluster_emotion']
        elif row['cluster_emotion'] == 'Neutral':
            return 'Neutral'  # Or any other default logic you prefer
        else:
            return 'Uncategorized'  # If cluster emotion is also not helpful

    # Regular handling for 'Sad/Angry' when feature emotion is not 'Uncategorized'
    elif row['valence_emotion'] == 'Sad/Angry':
        scores = {'Sad': 0, 'Angry': 0}
        if row['feature_emotion'] in ['Sad', 'Angry']:
            scores[row['feature_emotion']] += feature_score
        if row['cluster_emotion'] in ['Sad', 'Angry']:
            scores[row['cluster_emotion']] += cluster_score
        return max(scores, key=scores.get) if max(scores.values()) > 0 else 'Sad/Angry'

    # Regular voting for other cases
    votes = {'Happy': 0, 'Sad': 0, 'Angry': 0, 'Neutral': 0}
    votes[row['valence_emotion']] += valence_score
    if row['feature_emotion'] != 'Uncategorized':
        votes[row['feature_emotion']] += feature_score
    votes[row['cluster_emotion']] += cluster_score
    return max(votes, key=votes.get)

# Apply the function to each row in the DataFrame
df['combined_emotion'] = df.apply(combine_emotions, axis=1)


In [167]:
# filtered_df = df[df['combined_emotion'] == 'Uncategorized']
filtered_df = df

In [168]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
filtered_df['combined_emotion'] = label_encoder.fit_transform(filtered_df['combined_emotion'])

In [169]:
filtered_df = filtered_df.drop(['key_mode','cluster'], axis=1)
filtered_df


,tempo,key,mode,loudness,valence,valence_emotion,feature_emotion,cluster_emotion,combined_emotion
0nt1LAZjaTjrf30S8hdRDU,172.121,4,1,-6.773,0.271,Sad/Angry,Uncategorized,Neutral,2
0F02KChKwbcQ3tk4q1YxLH,137.130,7,1,-7.295,0.399,Neutral,Uncategorized,Neutral,2
4zWz6OmFpyXIgQCxAS5yQb,73.947,2,1,-6.207,0.397,Neutral,Uncategorized,Angry,2
4KJx4IOvCU2isprmXio9Xk,147.916,0,1,-10.306,0.345,Neutral,Uncategorized,Neutral,2
4hYHaROtjB6YREh6AgW3fd,135.977,0,1,-7.191,0.150,Sad/Angry,Uncategorized,Neutral,2
...,...,...,...,...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,139.941,1,1,-8.355,0.877,Happy,Happy,Angry,1
4NI8oSMRCJdpJPFux5G92u,127.845,1,1,-5.705,0.161,Sad/Angry,Uncategorized,Neutral,2
2tgIvnkdDSBYsNBPGeffLJ,122.831,2,1,-8.455,0.222,Sad/Angry,Uncategorized,Neutral,2
2gjE7iURty9r45rlgp6JT1,132.273,9,1,-8.061,0.181,Sad/Angry,Uncategorized,Neutral,2


In [170]:
filtered_df = filtered_df.drop(['valence_emotion', 'feature_emotion', 'cluster_emotion'], axis=1)
filtered_df.rename(columns={'combined_emotion': 'combinedemotion'}, inplace=True)
filtered_df

,tempo,key,mode,loudness,valence,combinedemotion
0nt1LAZjaTjrf30S8hdRDU,172.121,4,1,-6.773,0.271,2
0F02KChKwbcQ3tk4q1YxLH,137.130,7,1,-7.295,0.399,2
4zWz6OmFpyXIgQCxAS5yQb,73.947,2,1,-6.207,0.397,2
4KJx4IOvCU2isprmXio9Xk,147.916,0,1,-10.306,0.345,2
4hYHaROtjB6YREh6AgW3fd,135.977,0,1,-7.191,0.150,2
...,...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,139.941,1,1,-8.355,0.877,1
4NI8oSMRCJdpJPFux5G92u,127.845,1,1,-5.705,0.161,2
2tgIvnkdDSBYsNBPGeffLJ,122.831,2,1,-8.455,0.222,2
2gjE7iURty9r45rlgp6JT1,132.273,9,1,-8.061,0.181,2


In [171]:
scaler = StandardScaler()
features_to_scale = ['tempo', 'loudness', 'valence']
filtered_df[features_to_scale] = scaler.fit_transform(filtered_df[features_to_scale])

In [172]:
keys_encoded = pd.get_dummies(filtered_df['key'], prefix='key')
training_keys_columns = ['key_0', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11']
for column in training_keys_columns:
    if column not in keys_encoded.columns:
        keys_encoded[column] = False

keys_encoded = keys_encoded[training_keys_columns]

In [173]:
keys_encoded

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0nt1LAZjaTjrf30S8hdRDU,False,False,False,False,True,False,False,False,False,False,False,False
0F02KChKwbcQ3tk4q1YxLH,False,False,False,False,False,False,False,True,False,False,False,False
4zWz6OmFpyXIgQCxAS5yQb,False,False,True,False,False,False,False,False,False,False,False,False
4KJx4IOvCU2isprmXio9Xk,True,False,False,False,False,False,False,False,False,False,False,False
4hYHaROtjB6YREh6AgW3fd,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,False,True,False,False,False,False,False,False,False,False,False,False
4NI8oSMRCJdpJPFux5G92u,False,True,False,False,False,False,False,False,False,False,False,False
2tgIvnkdDSBYsNBPGeffLJ,False,False,True,False,False,False,False,False,False,False,False,False
2gjE7iURty9r45rlgp6JT1,False,False,False,False,False,False,False,False,False,True,False,False


In [174]:
filtered_df = pd.concat([filtered_df, keys_encoded], axis=1)
filtered_df

,tempo,key,mode,loudness,valence,combinedemotion,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0nt1LAZjaTjrf30S8hdRDU,1.652813,4,1,0.286801,-0.512097,2,False,False,False,False,True,False,False,False,False,False,False,False
0F02KChKwbcQ3tk4q1YxLH,0.474034,7,1,0.105605,0.178164,2,False,False,False,False,False,False,False,True,False,False,False,False
4zWz6OmFpyXIgQCxAS5yQb,-1.654480,2,1,0.483269,0.167379,2,False,False,True,False,False,False,False,False,False,False,False,False
4KJx4IOvCU2isprmXio9Xk,0.837393,0,1,-0.939565,-0.113040,2,True,False,False,False,False,False,False,False,False,False,False,False
4hYHaROtjB6YREh6AgW3fd,0.435191,0,1,0.141706,-1.164609,2,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,0.568731,1,1,-0.262339,2.755858,1,False,True,False,False,False,False,False,False,False,False,False,False
4NI8oSMRCJdpJPFux5G92u,0.161240,1,1,0.657522,-1.105290,2,False,True,False,False,False,False,False,False,False,False,False,False
2tgIvnkdDSBYsNBPGeffLJ,-0.007672,2,1,-0.297051,-0.776338,2,False,False,True,False,False,False,False,False,False,False,False,False
2gjE7iURty9r45rlgp6JT1,0.310411,9,1,-0.160287,-0.997437,2,False,False,False,False,False,False,False,False,False,True,False,False


In [175]:
for column in keys_encoded.columns:
    filtered_df[f"{column}_mode_interaction"] = filtered_df[column] * filtered_df['mode']

In [176]:
filtered_df['tempo_loudness'] = filtered_df['tempo'] * filtered_df['loudness']
filtered_df

,tempo,key,mode,loudness,valence,combinedemotion,key_0,key_1,key_2,key_3,...,key_3_mode_interaction,key_4_mode_interaction,key_5_mode_interaction,key_6_mode_interaction,key_7_mode_interaction,key_8_mode_interaction,key_9_mode_interaction,key_10_mode_interaction,key_11_mode_interaction,tempo_loudness
0nt1LAZjaTjrf30S8hdRDU,1.652813,4,1,0.286801,-0.512097,2,False,False,False,False,...,0,1,0,0,0,0,0,0,0,0.474028
0F02KChKwbcQ3tk4q1YxLH,0.474034,7,1,0.105605,0.178164,2,False,False,False,False,...,0,0,0,0,1,0,0,0,0,0.050060
4zWz6OmFpyXIgQCxAS5yQb,-1.654480,2,1,0.483269,0.167379,2,False,False,True,False,...,0,0,0,0,0,0,0,0,0,-0.799559
4KJx4IOvCU2isprmXio9Xk,0.837393,0,1,-0.939565,-0.113040,2,True,False,False,False,...,0,0,0,0,0,0,0,0,0,-0.786785
4hYHaROtjB6YREh6AgW3fd,0.435191,0,1,0.141706,-1.164609,2,True,False,False,False,...,0,0,0,0,0,0,0,0,0,0.061669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,0.568731,1,1,-0.262339,2.755858,1,False,True,False,False,...,0,0,0,0,0,0,0,0,0,-0.149200
4NI8oSMRCJdpJPFux5G92u,0.161240,1,1,0.657522,-1.105290,2,False,True,False,False,...,0,0,0,0,0,0,0,0,0,0.106019
2tgIvnkdDSBYsNBPGeffLJ,-0.007672,2,1,-0.297051,-0.776338,2,False,False,True,False,...,0,0,0,0,0,0,0,0,0,0.002279
2gjE7iURty9r45rlgp6JT1,0.310411,9,1,-0.160287,-0.997437,2,False,False,False,False,...,0,0,0,0,0,0,1,0,0,-0.049755


In [177]:
from sklearn.preprocessing import PolynomialFeatures

X = filtered_df[['tempo', 'loudness', 'key_0_mode_interaction', 'key_1_mode_interaction',
       'key_2_mode_interaction', 'key_3_mode_interaction',
       'key_4_mode_interaction', 'key_5_mode_interaction',
       'key_6_mode_interaction', 'key_7_mode_interaction',
       'key_8_mode_interaction', 'key_9_mode_interaction',
       'key_10_mode_interaction', 'key_11_mode_interaction',]]
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(['tempo', 'loudness', 'key_0_mode_interaction', 'key_1_mode_interaction',
       'key_2_mode_interaction', 'key_3_mode_interaction',
       'key_4_mode_interaction', 'key_5_mode_interaction',
       'key_6_mode_interaction', 'key_7_mode_interaction',
       'key_8_mode_interaction', 'key_9_mode_interaction',
       'key_10_mode_interaction', 'key_11_mode_interaction',]))
df_poly = pd.concat([filtered_df.reset_index(drop=True), poly_df], axis=1)

In [178]:
# Calculate quantiles for tempo
tempo_quantiles = filtered_df['tempo'].quantile([0.25, 0.5, 0.75])

# Use the quantiles to define bins
tempo_bins = [-np.inf, tempo_quantiles[0.25], tempo_quantiles[0.5], tempo_quantiles[0.75], np.inf]
filtered_df['tempo_quantile_bins'] = pd.cut(filtered_df['tempo'], bins=tempo_bins, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Calculate quantiles for loudness
loudness_quantiles = filtered_df['loudness'].quantile([0.25, 0.5, 0.75])

# Use the quantiles to define bins
loudness_bins = [-np.inf, loudness_quantiles[0.25], loudness_quantiles[0.5], loudness_quantiles[0.75], np.inf]
filtered_df['loudness_quantile_bins'] = pd.cut(filtered_df['loudness'], bins=loudness_bins, labels=['Q1', 'Q2', 'Q3', 'Q4'])


In [179]:
filtered_df = pd.get_dummies(filtered_df, columns=['tempo_quantile_bins', 'loudness_quantile_bins'])

In [180]:
filtered_df = filtered_df.drop(['combinedemotion'],axis=1)
filtered_df.columns

Index(['tempo', 'key', 'mode', 'loudness', 'valence', 'key_0', 'key_1',
       'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9',
       'key_10', 'key_11', 'key_0_mode_interaction', 'key_1_mode_interaction',
       'key_2_mode_interaction', 'key_3_mode_interaction',
       'key_4_mode_interaction', 'key_5_mode_interaction',
       'key_6_mode_interaction', 'key_7_mode_interaction',
       'key_8_mode_interaction', 'key_9_mode_interaction',
       'key_10_mode_interaction', 'key_11_mode_interaction', 'tempo_loudness',
       'tempo_quantile_bins_Q1', 'tempo_quantile_bins_Q2',
       'tempo_quantile_bins_Q3', 'tempo_quantile_bins_Q4',
       'loudness_quantile_bins_Q1', 'loudness_quantile_bins_Q2',
       'loudness_quantile_bins_Q3', 'loudness_quantile_bins_Q4'],
      dtype='object')

In [181]:
predictions = load_model.predict(filtered_df)

In [182]:
print(predictions)

[2 2 2 2 3 1 2 0 3 2 2 2 2 2 2 2 2 2 2 2 1 2 2 3 2 2 2 2 3 3 2 2 2 2 2 2 2
 2 0 1 2 2 2 3 2 2 2 1 2 2 1 2 2 2 2 2 2 0 2 2 3 1 2 1 2 2 2 2 2 2 1 1 2 2
 2 2 1 2 0 2 2 2 3 2 2 1 1 1 2 2 2 2 2 1 1 1 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2
 2 2 2 2 2 1 1 2 2 1 2 0 2 2 1 2 1 0 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 2 1
 1 2 2 1 1 2 2 2 2 2 2 2 1 1 2 2 0 2 0 1 2 2 2 1 2 1 2 2 2 3 2 2 1 2 2 2 2
 2 2 2 2 2 0 1 2 1 0 2 2 2 2 1 3 2 2 2]


In [184]:
emotion_mapping = {0: 'Angry', 1: 'Happy', 2: 'Neutral', 3: 'Sad'}
emotion_labels = [emotion_mapping[pred] for pred in predictions]

In [185]:
new_data = df
new_data.loc[:, 'combined_emotion'] = emotion_labels
new_data

,tempo,key,mode,loudness,valence,key_mode,valence_emotion,feature_emotion,cluster,cluster_emotion,combined_emotion
0nt1LAZjaTjrf30S8hdRDU,172.121,4,1,-6.773,0.271,4.0 Major,Sad/Angry,Uncategorized,2,Neutral,Neutral
0F02KChKwbcQ3tk4q1YxLH,137.130,7,1,-7.295,0.399,7.0 Major,Neutral,Uncategorized,2,Neutral,Neutral
4zWz6OmFpyXIgQCxAS5yQb,73.947,2,1,-6.207,0.397,2.0 Major,Neutral,Uncategorized,0,Angry,Neutral
4KJx4IOvCU2isprmXio9Xk,147.916,0,1,-10.306,0.345,0.0 Major,Neutral,Uncategorized,2,Neutral,Neutral
4hYHaROtjB6YREh6AgW3fd,135.977,0,1,-7.191,0.150,0.0 Major,Sad/Angry,Uncategorized,2,Neutral,Sad
...,...,...,...,...,...,...,...,...,...,...,...
26so7ajU82ioMhfYQTuIhR,139.941,1,1,-8.355,0.877,1.0 Major,Happy,Happy,0,Angry,Happy
4NI8oSMRCJdpJPFux5G92u,127.845,1,1,-5.705,0.161,1.0 Major,Sad/Angry,Uncategorized,2,Neutral,Sad
2tgIvnkdDSBYsNBPGeffLJ,122.831,2,1,-8.455,0.222,2.0 Major,Sad/Angry,Uncategorized,2,Neutral,Neutral
2gjE7iURty9r45rlgp6JT1,132.273,9,1,-8.061,0.181,9.0 Major,Sad/Angry,Uncategorized,2,Neutral,Neutral


In [175]:
def get_album_details(album_id, albums_data):
    for album in albums_data:
        if album['albumID'] == album_id:
            return album
    return None

def get_artist_details(artist_id, all_artists):
    for artist in all_artists:
        if artist['artistID'] == artist_id:
            return artist
    return None

In [174]:
all_artists = []

for country_tracks in top_tracks_by_country_filtered.values():
    for track_info in country_tracks:
        for artist in track_info['track']['artists']:
            artist_data = {
                'artistID': artist['id'],
                'name': artist['name']
            }
            # Check for duplicates
            if not any(a['artistID'] == artist_data['artistID'] for a in all_artists):
                all_artists.append(artist_data)

albums_data = []

for country_tracks in top_tracks_by_country_filtered.values():
    for track_info in country_tracks:
        album = track_info['track']['album']
        if album['artists']:
            album_data = {
                'albumID': album['id'],
                'title': album['name'],
                'artistID': album['artists'][0]['id'],
                'releaseDate': album['release_date'],
                'coverArtURL': album['images'][0]['url'] if album['images'] else None
            }
        else:
            print(album['name'])
        if album_data not in albums_data:
            albums_data.append(album_data)

In [176]:
track_info = {}
album_info = {}
artist_info = {}

for track_id in new_data.index:
    track_details = tracks_data.get(track_id, None)
    if track_details:
        album_details = get_album_details(track_details['albumID'], albums_data)
        artist_details = get_artist_details(track_details['artistID'], all_artists)
        if track_id not in track_info:
            track_info[track_id] = {
                'trackID': track_id,
                'title': track['name'],
                'artistID': track['artists'][0]['id'], 
                'albumID': track['album']['id'],
                'duration': track['duration_ms'],
                'releaseDate': track['album']['release_date'],
                'spotifyURL': track['external_urls']['spotify'],
                'popularity_by_country': {country_code: track['popularity']}
            }
        else:
            track_info[track_id]['popularity_by_country'][country_code] = track['popularity']
        
        if album_details and album_details['albumID'] not in album_info:
            album_info[album_details['albumID']] = {
                'albumID': album_details['albumID'],
                'title': album_details['title'],
                'artistID': album_details['artistID'],
                'releaseDate': album_details['releaseDate'],
                'coverArtURL': album_details['coverArtURL']
            }

        if artist_details and artist_details['artistID'] not in artist_info:
            artist_info[artist_details['artistID']] = {
                'artistID': artist_details['artistID'],
                'name': artist_details['name']
            }


In [178]:
import psycopg2

def insert_album(albumid, title, artistid, releasedate, coverarturl):
    conn = None
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(
            dbname="sentirhy",
            user="owner",
            password="oaPBeAzranUQ",
            host="104.168.28.119"
        )
        cursor = conn.cursor()

        # Insert artist data
        cursor.execute("""
            INSERT INTO sentirhy.albums (albumid, title, artistid, releasedate, coverarturl)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (albumid) DO NOTHING; 
        """, (albumid, title, artistid, releasedate, coverarturl))

        conn.commit()
        cursor.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def insert_artist(artist_id, name):
    conn = None
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(
            dbname="sentirhy",
            user="owner",
            password="oaPBeAzranUQ",
            host="104.168.28.119"
        )
        cursor = conn.cursor()

        # Insert artist data
        cursor.execute("""
            INSERT INTO sentirhy.artists (artistID, name)
            VALUES (%s, %s)
            ON CONFLICT (artistID) DO NOTHING; 
        """, (artist_id, name))

        conn.commit()
        cursor.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def insert_audio(trackid, tempo, key, mode, loudness, valence, valenceemotion, featureemotion, clusteremotion, combinedemotion):
    conn = None
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(
            dbname="sentirhy",
            user="owner",
            password="oaPBeAzranUQ",
            host="104.168.28.119"
        )
        cursor = conn.cursor()

        # Insert artist data
        cursor.execute("""
            INSERT INTO sentirhy.audiofeats (trackid, tempo, key, mode, loudness, valence, valenceemotion, featureemotion, clusteremotion, combinedemotion)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (trackid) DO NOTHING; 
        """, (trackid, tempo, key, mode, loudness, valence, valenceemotion, featureemotion, clusteremotion, combinedemotion))

        conn.commit()
        cursor.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def insert_track(trackid, title, artistid, albumid, duration,  releasedate, spotifyurl):
    conn = None
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(
            dbname="sentirhy",
            user="owner",
            password="oaPBeAzranUQ",
            host="104.168.28.119"
        )
        cursor = conn.cursor()

        # Insert artist data
        cursor.execute("""
            INSERT INTO sentirhy.tracks (trackid, title, artistid, albumid, duration, releasedate, spotifyurl)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (trackid) DO NOTHING; 
        """, (trackid, title, artistid, albumid, duration, releasedate, spotifyurl))

        conn.commit()
        cursor.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [179]:
for index, row in new_data.iterrows():
    insert_audio(index, row['tempo'], row['key'], row['mode'], row['loudness'], row['valence'], row['valence_emotion'], row['feature_emotion'], row['cluster_emotion'], row['combined_emotion'])

In [187]:
for artist in artist_info:
    insert_artist(artist_info[artist]['artistID'], artist_info[artist]['name'])

In [189]:
for track in track_info:
    insert_track(track_info[track]['trackID'], track_info[track]['title'], track_info[track]['artistID'], track_info[track]['albumID'], track_info[track]['duration'], track_info[track]['releaseDate'], track_info[track]['spotifyURL'])

In [190]:
for album in album_info:
    insert_album(album_info[album]['albumID'],album_info[album]['title'], album_info[album]['artistID'], album_info[album]['releaseDate'], album_info[album]['coverArtURL'])